<a href="https://colab.research.google.com/github/Duke-BC-DL-for-Energy-Infrastructure/Colab-Notebooks-for-Training-Models/blob/main/Transmission-Towers/ALL_Domains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!git clone https://github.com/Duke-BC-DL-for-Energy-Infrastructure/yolov3.git  # clone our fork of the ultralytics repo

In [ ]:
!mkdir -p data/

# get our data and unzip it
!wget -nc -O data/images.zip https://duke.box.com/shared/static/c319qsyg9ohzetpens2091t8b9ikc4nv.zip
!wget -nc -O data/labels.zip https://duke.box.com/shared/static/on1ntto9kkljh24dla15gsavi0fnszem.zip

!unzip -n -q data/images.zip -d data
!unzip -n -q data/labels.zip -d data

In [ ]:
# get modified cfg file that works for 1 class
!rm -rf yolov3/cfg 

!wget -nc -O yolov3/cfg.zip https://duke.box.com/shared/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
!unzip -n -q yolov3/cfg.zip -d yolov3

In [ ]:
!rm -rf yolov3/data

# load in the data.zip file from the box
!wget -nc -O yolov3/baseline.zip https://duke.box.com/shared/static/x8uptj03kr36k5hwggjwga3vpjqaau9p.zip
!unzip -n -q yolov3/baseline.zip -d yolov3

In [ ]:
%pwd

In [ ]:
%cd yolov3

In [11]:
!python3 train.py --cfg cfg/yolov3-spp.cfg --data baseline/baseline.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data baseline/baseline.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [9]:
# Create folder for validation dataset, so that we can run detect.py on it

VAL_PATH = 'baseline/val_img_paths.txt' # Provide path to .txt file for validation set

from shutil import copyfile
%mkdir ../data/val

with open(VAL_PATH, 'r') as f:
  val_imgs =  [path.replace('\n', '') for path in f.readlines()]

for img in val_imgs:
  # copy image to val folder
  copyfile(img, img.replace('images', 'val'))

In [ ]:
# DETECT using last.pt
!python3 detect.py --names baseline/wnd.names --weights weights/last.pt --source ../data/val --img-size 608